In [94]:
import os
from bs4 import BeautifulSoup
import re
import pandas as pd

In [95]:
def unfuck(raw_html):
    raw_html = str(raw_html)
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return cleantext.replace('\t', '').replace('\n', '').strip(' ')

In [99]:
predmety = {
    "AJ": "Anglický jazyk",
    "CJ": "Český jazyk a literatura",
    "FJ": "Francouzský jazyk",
    "MA": "Matematika",
    "NJ": "Německý jazyk",
    "RJ": "Ruský jazyk",
    "SJ": "Španělský jazyk"
}

typ = {
    "DT": "Didaktický test",
    "PP": "Písemná práce",
    "UZ": "Ústní zkouška"
}
    
obory = {
    "GY4": "Gymnázium 4 leté",
    "GY6": "Gymnázium 6 leté",
    "GY8": "Gymnázium 8 leté",
    "LYC": "Lyceum",
    "S-EK": "SOŠ ekonomické",
    "S-HP": "SOŠ hotelové a podnikatelské",
    "S-HU": "SOŠ pedagogické a humanitní",
    "S-T1": "SOŠ technické 1",
    "S-T2": "SOŠ technické 2",
    "S-UM": "SOŠ umělecké",
    "S-ZD": "SOŠ zdravotnické",
    "S-ZE": "SOŠ zemědělské",
    "U-OS": "SOU ostatní",
    "N-OS": "SOU ostatní (nástavby)",
    "U-TE": "SOU technické",
    "N-TE": "SOU technické (nástavby)"
}

In [110]:
out = []
for file in os.listdir('./scratch/'):
    soup = BeautifulSoup(open('./scratch/' + file, encoding='utf-8'), 'html.parser')
    f = file.split('_')
    rok = f[0]
    predmet = predmety[f[1]]
    zkouska = typ[f[2]]
    skola_obor = obory[f[3].split('.')[0]]
    
    ks = ['tmp', 'REDIZO', 'nazev', 'typ']
    keys = list(map(unfuck, soup.find('thead').find_all('th')[2:]))
    ks.extend(keys)
    for row in soup.find_all('tr'):
        x = row.find('td', text=re.compile('REDIZO'))
        if x != None:
            tmp = []
            rw = x.parent.find_all('td')
            for r in rw:
                tmp.append(r.text)
            d = dict(zip(ks, tmp))
            d.update({
                'rok': rok,
                'predmet': predmet,
                'typ_zkousky': zkouska,
                'skola_obor': skola_obor
            })
            out.append(d)

In [111]:
dta = pd.DataFrame.from_dict(out)

In [112]:
def nabodenicka_pdp(val):
    return val.replace(',', '.')

In [113]:
dta = dta.applymap(nabodenicka_pdp)

In [114]:
ne = ['nazev', 'tmp', 'typ', 'rok', 'predmet', 'typ_zkousky', 'skola_obor']
for col in dta.columns:
    if col not in  ne:
        dta[col] = pd.to_numeric(dta[col], errors = 'coerce')

In [115]:
dta.drop_duplicates(inplace=True)

In [116]:
dta.to_csv('./data/data_dump.csv', encoding='utf-8', index=False)